In [ ]:
pip install orion-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing Lib


In [ ]:
from orion import Orion
import yfinance as yf
import pandas as pd
import numpy as np

#Getting Raw Data

In [ ]:
#Importig Apple Stock data from yfinance
start = "2015-01-01"
end = "2020-01-01"
appl_df = yf.download("AAPL", start = start,end = end)
print(appl_df)
#Getting Date and Adjusted Closing Price of Apple Stock
appl_df_1 = appl_df.reset_index()[['Date','Adj Close']]
aapl = yf.Ticker('AAPL')


#Making table with Date converted to timestamp in nanoseconds
tmdata = pd.Series(appl_df_1['Date'].values.astype(np.int64))

#Merging table with timestamp with original Table
appl_df_2 = appl_df_1.merge(tmdata.rename('Timestamp'), how = 'inner', left_index= True, right_index = True)

#Dropping Date
appl_df_2.drop('Date', axis=1, inplace= True)

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2015-01-02  27.847500  27.860001  26.837500  27.332500  24.644020  212818400
2015-01-05  27.072500  27.162500  26.352501  26.562500  23.949755  257142000
2015-01-06  26.635000  26.857500  26.157499  26.565001  23.952013  263188400
2015-01-07  26.799999  27.049999  26.674999  26.937500  24.287876  160423600
2015-01-08  27.307501  28.037500  27.174999  27.972500  25.221066  237458000
2015-01-09  28.167500  28.312500  27.552500  28.002501  25.248114  214798000
2015-01-12  28.150000  28.157499  27.200001  27.312500  24.625982  198603200
2015-01-13  27.857500  28.200001  27.227501  27.555000  24.844633  268367600
2015-01-14  27.260000  27.622499  27.125000  27.450001  24.749960  195826400
2015-01-15  27.500000  27.514999  26.665001  26.705000  24.078238  240056000
2015-01

In [ ]:
hist = aapl.history(start=start, end=end)
appl_df_3 = appl_df_2.reindex(columns = ['Timestamp', 'Adj Close'])
appl_final = appl_df_3.rename(columns={'Timestamp': 'timestamp', 'Adj Close': 'value'})
appl_final['value'] = appl_final['value'] /appl_final['value'].abs().max()
appl_final['timestamp'] = appl_final['timestamp'].div(1000000000)

In [ ]:
hyperparameters = {
    'orion.primitives.tadgan.TadGAN#1': {
        'epochs': 5
        }
}

orion = Orion(
    pipeline='tadgan',
    hyperparameters=hyperparameters
)


In [ ]:
orion.fit(appl_final)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected

Epoch: 1/5, [Dx loss: [-1.8083769   0.41599083 -2.479624    0.02552564]] [Dz loss: [-1.3510131   1.2187536  -3.9475656   0.13777953]] [G loss: [9.873273   2.5276413  6.6265054  0.07191282]]
Epoch: 2/5, [Dx loss: [-1.8552474  -2.509784    0.5345722   0.01199652]] [Dz loss: [ 2.4497724   1.0017264  -0.02463207  0.14726786]] [G loss: [ 0.00759935 -0.5549236   0.05017689  0.0512346 ]]
Epoch: 3/5, [Dx loss: [-1.9002211  -4.9338055   2.908676    0.01249051]] [Dz loss: [2.6416934  0.5077051  0.29250085 0.18414864]] [G loss: [-2.679525   -2.9299831  -0.24802467  0.04984841]]
Epoch: 4/5, [Dx loss: [-1.9170754  -7.491761    5.447972    0.01267105]] [Dz loss: [2.2396548  0.20208047 0.50523275 0.15323426]] [G loss: [-5.4659224  -5.486709   -0.4918123   0.05125998]]
Epoch: 5/5, [Dx loss: [-1.9163022  -7.0793266   5.0315514   0.01314736]] [Dz loss: [ 1.7781478  -0.24255207  0.7317606   0.1288939 ]] [G loss: [-5.1436234  -5.0291486  -0.6378403   0.05233657]]


In [ ]:
anomalies = orion.detect(appl_final)

In [ ]:
anomalies['start'] = pd.to_datetime(anomalies['start'], unit='s')
anomalies['end'] = pd.to_datetime(anomalies['end'], unit='s')
anomalies

,start,end,severity
0,2018-08-02 06:00:00,2018-08-18 18:00:00,0.140713


In [ ]:
anomalies_hist = [aapl.history(start=anomalies['start'].iloc[i], end=anomalies['end'].iloc[i]) for i in range(anomalies.shape[0])]
anomalies_hist

[                 Open       High        Low      Close     Volume  Dividends  \
 Date                                                                           
 2018-08-02  48.141626  50.013721  48.086424  49.776108  249616000     0.0000   
 2018-08-03  49.689692  50.100114  49.317672  49.920105  133789600     0.0000   
 2018-08-06  49.922507  50.222522  49.699297  50.179321  101701600     0.0000   
 2018-08-07  50.239336  50.282537  49.624902  49.708908  102349600     0.0000   
 2018-08-08  49.454483  49.876903  49.087265  49.742496   90102000     0.0000   
 2018-08-09  50.289731  50.349734  49.730502  50.133724   93970400     0.0000   
 2018-08-10  49.943449  50.362536  49.777260  49.984394   98444800     0.1825   
 2018-08-13  50.413117  50.808117  50.025342  50.307140  103563600     0.0000   
 2018-08-14  50.617832  50.714172  50.160208  50.519081   82992000     0.0000   
 2018-08-15  50.391436  50.757535  50.177076  50.637108  115230400     0.0000   
 2018-08-16  51.000802  51.4

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=hist.index,y=hist['Close'],name='Price'),secondary_y=False)
for i in range(len(anomalies_hist)):
  fig.add_trace(go.Scatter(x=anomalies_hist[i].index.values,y=anomalies_hist[i]['Close'],name='Anomaly ' + str(i)),secondary_y=False)
fig.show()